In [2]:
import torch
from torch import nn # contains all of pytorchs building block.
import matplotlib.pyplot as plt

torch.__version__

'2.6.0+cpu'

## Data (preparing and loading)

Can be anything in ml

* Excels speadsheet
* Images and videos
* Audio 
* DNA


Ml it's a game in two parts
* Get the data into a numberical representation
* Build a model to learn the patterns

In [3]:
# Create some "know" data using LINEAR REGRESSION



# Create *know* parameters
weight = 0.7 # B
bias = 0.3 # A

# Create
X = torch.arange(0, 1, 0.02).unsqueeze(dim = 1)
y = weight * X + bias

print(X[:10]),print(y[:10])

tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800]])
tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])


(None, None)

In [4]:
print(len(X), len(y))

50 50


## Spliting data into training and test sets

In [5]:
# Create a train/test split

train_split = int(0.8 * (len(X)))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
print(len(X_train)), print(len(y_train)), print(len(X_test)), print(len(y_test))

40
40
10
10


(None, None, None, None)

### PyTorch model builds essentials
* torch.nn -> contains all of the buildings for computuional graphs
* torch.nn.Parameter -> what parameters should our model try and learn
* torch.nn.Module -> the base class for all neural netowrks
* torch.optim -> this where the optimizers in Pytorch live, helo with gradient descent
* def forward -> Defines what happens in the forward computation

### Making predictions using `torch.inference_mode()`

In [6]:
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.weight = nn.Parameter(torch.randn(1,))
        self.bias = nn.Parameter(torch.randn(1,))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weight * x + self.bias

torch.manual_seed(42)
model_0 = LinearRegression()

with torch.inference_mode(): # or we can use the .no_grad()
    y_preds = model_0(X_test)
print(y_preds)
print(y_test)



tensor([[0.3982],
        [0.4049],
        [0.4116],
        [0.4184],
        [0.4251],
        [0.4318],
        [0.4386],
        [0.4453],
        [0.4520],
        [0.4588]])
tensor([[0.8600],
        [0.8740],
        [0.8880],
        [0.9020],
        [0.9160],
        [0.9300],
        [0.9440],
        [0.9580],
        [0.9720],
        [0.9860]])


### Train model
in our case, our predictions be so poor, but how we can know that's it's a poor or better predictions, we use the loss funtions to meansure that and after use the optimizer to adjust these values, in these case we gonna use the MAE loss functiom (L1loss)

In [7]:
# loss function

loss_fn = nn.L1Loss()


# optimizer (stochastic gradient descent)
optimizer = torch.optim.SGD(params=model_0.parameters(), lr =0.01) # lr it's the learning rate, how our pred it's 4 numbers after the ., when we use 0.01 it's 2 numbers, so a medium chagnes, if we put 0.1 will update a big changes



### Building a training and testing loop
Thing need in a training loop:
0. Loop trough the data
1. Forward pass to make predictions
2. Calculate loss, compare forward pass to the true labels
3. Optimizer zero grad
4. Loss backward (backpropagation)
5. Optimizer step (**gradient descent**)


In [14]:
# 0 loop trought the data
epochs = 10000 # épocas

for epoch in range(epochs):
    model_0.train()  # Set to the train mode
    y_pred = model_0(X_train)
    loss = loss_fn(y_pred, y_train)  # Calcule the loss function (predicions with the real labels)]
    optimizer.zero_grad() # The gradient turns 0 to calculate new grad
    loss.backward() # Calculate the gradient
    optimizer.step() # Update the parameters
    if epoch % 1000 == 0:
        print(f"A loss function foi de: {loss:.4f}")

model_0.eval()
print(f"Os novos parametros são:")
for nome, parametro in model_0.named_parameters():
    print(f"Nome: {nome}, {parametro}")



A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
A loss function foi de: 0.0089
Os novos parametros são:
Nome: weight, Parameter containing:
tensor([0.6990], requires_grad=True)
Nome: bias, Parameter containing:
tensor([0.3093], requires_grad=True)


In [15]:
# testing code
model_0.eval()
with torch.inference_mode():
    y_preds_new = model_0(X_test)
    loss = loss_fn(y_preds_new, y_test)
print(f"The test loss is: {loss:.4f}")


The test loss is: 0.0084


### Saving a model in PyTorch
There are three main methods you should about for saving and loading models in torch

1. `torch.save()` allow you save a pytorch object in python's pickle format
2. `torch.load()` allows you load a saved PyTorch object
3. `torch.nn.Module.load_state_dict()` allows you to load a model saved state dictionary

In [22]:
# Saving

from pathlib import Path

# Create models directory
MODEL_PATH = Path("Models")
MODEL_PATH.mkdir()

# Create models save path

MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save in a state dict
print(f"Save path in {MODEL_SAVE_PATH}")
torch.save(model_0.state_dict(), MODEL_SAVE_PATH)

Save path in Models\01_pytorch_workflow_model_0.pth
